# Implementación del agente con Monte Carlo

In [ ]:
import numpy as np
rng = np.random.default_rng()

class MonteCarloAgent:
    '''
    Monte-Carlo agent.
    '''

    def __init__(self, env, gamma, epsilon, alpha):
        '''
        Constructor.

        @param env: the environment
        @param gamma: the discount factor
        @param epsilon: the probability to explore
        @param alpha: the learning rate
        '''
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.alpha = alpha


        # Q_table
        self.Q = {}
        

    def act(self, state):
        # Convertir la lista a una tupla usando hash para hacerla hashable
        state_key = hash(str(state))
        
        # Acceder al valor Q correspondiente al estado
        q_values = self.Q.get(state_key, np.zeros(self.env.action_space.n))
        action = np.argmax(q_values)

        if rng.uniform() < self.epsilon:
            action = rng.choice(self.env.action_space.n)
        
        action_to_names = {
            0: 'RIGHT',
            1: 'DOWN',
            2: 'LEFT',
            3: 'UP',
            4: 'BOMB',
            5: 'WAIT',
        }

        #print("Action: ", action_to_names[action])
        
        return action
    
    def update(self, episode):

        return_episode = 0.0

        # iterate backwards over the episode

        for state, action, reward in reversed(episode):
            
            state_key = hash(str(state))


            return_episode = reward + self.gamma * return_episode
            q_values = self.Q.get(state_key, np.zeros(self.env.action_space.n))
            q_values[action] = q_values[action] + self.alpha * (return_episode - q_values[action])
            self.Q[state_key] = q_values


    def train(self, nb_episode, recorder=False):
        ''''
        Runs the agent on the environment
        '''
        returns = []
        steps = []

        for episode in range(nb_episode):
            print("Episodio nº: ", episode)
            # reset
            state, info = self.env.reset()
            return_episode = 0.0
            nb_steps = 0
            done = False

            transition = []

            while not done:
                # act
                action = self.act(state)

                # step
                next_state, reward,terminal, truncated, info = self.env.step(action)

                # update
                transition.append((state, action, reward))

                state = next_state

                done = terminal or truncated
                nb_steps += 1

                return_episode += reward 

            self.update(transition)

            # Store info
            returns.append(return_episode)
            steps.append(nb_steps)
        
        return returns,steps

# Resultados obtenidos con Monte Carlo

### Recompensa
En esta entrega, los valores para las recompensas se plantearon de la siguiente manera:
* Colocar una bomba: +10pts
* Romper una caja: +7pts
* Eliminar un enemigo: +5pts
* Llegar al objetivo: +100pts
* Morir por una explosión: -100pts
* Morir por un enemigo: -100pts
* Vagar luego de destruir la caja donde se encuentra el objetivo: -5pts

Además para poder entrenar al agente probamos diferentes escenarios, variando las dimensiones de la grilla, la cantidad de cajas rompibles y la cantidad de enemigos. Además experimentamos con diferentes valores para las variables gamma, epsilon y alfa

En base a lo anteriormente mencionado, se obtuvieron los siguientes resultados:
- Dimensiones de la grilla: 5x5
- Cantidad cajas rompibles: 1
- Cantidad de enemigos: 1 vertical
- Gamma: 0,9
- Epsilon: 0,5
- Alfa: 0,5

<div><img src="https://i.ibb.co/09G1hMz/Captura-de-Pantalla-2023-11-20-a-la-s-23-29-38.png" width=550/></div>

- Dimensiones de la grilla: 5x5
- Cantidad cajas rompibles: 1
- Cantidad de enemigos: 1 vertical
- Gamma: 0,9
- Epsilon: 0,9
- Alfa: 0,5

<div><img src="https://i.ibb.co/3Sgs0xM/Captura-de-Pantalla-2023-11-20-a-la-s-23-34-44.png" width=550/></div>


### Después de experimentar con diferenetes entradas se llega a ciertas conclusiones con respecto a los valores que pueden tomar gamma, epsilon y alfa. Los valores eleguidos son:
- Gamma: 1,0
- Epsilon: 0,1
- Alfa: 0,4

### El agente al tener estos valores tiene las siguientes carácteristicas: al tener un gran valor de gamma, tenemos que el agente valora tanto la recompensa actual como la recompensa futura, un valor bajo de epsilon nos dice que tenemos un agente más conservador, el cual, explota más de lo que explora y al tener un valor equilibrado de alfa tenemos que el ajuste de las estimaciones no serán ni muy rápidas ni muy lentas.

### Aquí podemos ver un ejemplo con las siguientes carácteristicas:
- Dimensiones de la grilla: 8x8
- Cantidad cajas rompibles: 6
- Cantidad de enemigos: 5 vertical, 5 horizontal
- Gamma: 1,0
- Epsilon: 0,1
- Alfa: 0,4

<div><img src="https://i.ibb.co/qjt7n4G/Captura-de-Pantalla-2023-11-21-a-la-s-02-43-49.png" width=550/></div>

### Con esto podemos ver que tenemos más recompensas positivas, sin embargo, todavía tenemos recompensas negativas altas. Es por esto que a partir de estos valores, hay que empezar a ver las limitantes: dimensiones de la grilla, cantidad de cajas y cantidad de enemigos.
### Siguiendo con el ejemplo anterior, pero cambiando la cantidad de cajas indestructibles a 16 fijas, tenemos lo siguiente:
<div><img src="https://i.ibb.co/BZ4bvp7/Captura-de-Pantalla-2023-11-21-a-la-s-05-22-32.png" width=550/></div>


### De esto podemos concluir que la cantidad de cajas destructibles tiene que ser el doble del valor de la dimensión de la grilla, en este caso 2 x 8 = 16. Así que por convención se tomará este criterio para la cantidad de cajas.
### Ahora es necesario definir la cantidad de enemigos. Después de varias pruebas y manteniendo los parámetros anteriores, pero variando la cantidad de enemigos siempre con la misma cantidad de enemigos con movimiento horizontal y movimiento vertical, en este caso son 7 y 7, se obtuvieron los siguientes resultados:

 <div><img src="https://i.ibb.co/bHsCNNH/Captura-de-Pantalla-2023-11-21-a-la-s-03-46-53.png" width=550/></div>

 ### En este caso es aceptable tomar estos 7 enemigos verticales y 7 enemigos horizontales, pero este sería el límite y se va a definir como regla tomar una cantidad menor a las dimensiones de la grilla. Cuando se toma una mayor cantidad de enemigos, no se obtienen malos resultados, pero las barras del returns que es donde se ven reflejadas las recompensas, parten en valores menores a 0.

 ### Por último es necesario definir las dimensiones máximas. 
 ### Para esto se experimento con varias dimensiones pero finalmente, se determinó que las dimensiones más acertadas eran 15x15, con 30 cajas fijas, 14 enemigos verticlaes y 14 enemigos horizontales. Con estos valores la ejecución se realentiza bastante, pero todavía tiene una velocidad prudente y los resultados son los siguientes:

 <div><img src="https://i.ibb.co/svWHmVb/Captura-de-Pantalla-2023-11-21-a-la-s-05-52-41.png" width=550/></div>

  